In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkTutorial').master('local[*]').getOrCreate()

data = [
    ('Sanjay', 35),
    ('Ajay', 39),
    ('Vijay', 60)
]
df = spark.createDataFrame(data).toDF('Name', 'Age')
df.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/02 10:51:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+------+---+
|  Name|Age|
+------+---+
|Sanjay| 35|
|  Ajay| 39|
| Vijay| 60|
+------+---+



In [5]:
df = spark.read\
    .option('inferSchema', 'true')\
    .option('header', 'true')\
    .csv('/Users/snishad/SparkDemo/01-HelloSpark/data/sample.csv')

# df.select('*').show()

# df.select('Age', 'Gender', 'Country', 'state').show()

# select all females
df.where('Gender = "Female"').select('Timestamp').show()

+-------------------+
|          Timestamp|
+-------------------+
|2014-08-27 11:29:31|
|2014-08-27 11:31:50|
|2014-08-27 11:32:39|
+-------------------+



In [6]:
# SPARK RDD

# from pyspark import SparkConf, SparkContext

# conf = SparkConf().setMaster('local[*]').setAppName('HelloRDD')

# sc = SparkContext(conf=conf).getOrCreate()

sc = spark.sparkContext

rdd = sc.textFile('/Users/snishad/SparkDemo/02-HelloRDD/data/sample.csv')

rdd1 = rdd.flatMap(lambda x: x.split(','))

rdd2 = rdd1.map(lambda x: (x, 1))

rdd3 = rdd2.reduceByKey(lambda x, y: x + y)

rdd3.foreach(lambda x: print(x))




('37', 1)
('2014-08-27 11:29:31', 1)('"Female"', 3)

('"IL"', 2)
('"No"', 79)
('"United States"', 6)
('"Yes"', 45)('NA', 21)

('"Often"', 2)
('"6-25"', 3)
('"M"', 2)('"Not sure"', 2)

('"Rarely"', 2)('"Somewhat easy"', 1)

('"Some of them"', 4)
('"Don\'t know"', 18)
('"Maybe"', 9)('2014-08-27 11:29:44', 1)

('32', 1)
('2014-08-27 11:29:37', 1)
('44', 1)
('"Male"', 4)('"IN"', 1)

('"More than 1000"', 1)('"Somewhat difficult"', 3)

('"Canada"', 2)('2014-08-27 11:29:46', 1)

('"United Kingdom"', 1)('31', 2)

('2014-08-27 11:30:22', 1)('"26-100"', 1)

('"TX"', 1)('"100-500"', 2)

('"Never"', 2)('2014-08-27 11:31:22', 1)
('33', 1)

('"Sometimes"', 3)('"TN"', 1)

('2014-08-27 11:31:50', 1)
('"MI"', 1)
('35', 1)
('2014-08-27 11:32:05', 1)
('"1-5"', 2)
('"Very difficult"', 1)
('39', 1)
('2014-08-27 11:32:39', 1)
('42', 1)


In [7]:
# SPARK SQL

from pyspark.sql import *

df = spark.read.option('inferSchema', 'true').option('header', 'true').csv('/Users/snishad/SparkDemo/03-HelloSparkSQL/data/sample.csv')

df.createOrReplaceTempView('MyView')

count_df = spark.sql('select Country, count(1) as count from MyView where Age > 40 group by Country')

count_df.show()

23/05/02 10:51:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------+-----+
|      Country|count|
+-------------+-----+
|United States|    2|
+-------------+-----+



In [8]:
# SPARK SCHEMA DEMO

# csv
csv_df = spark.read.format('csv').option('inferSchema', 'true').option('header', 'true').load('/Users/snishad/SparkDemo/04-SparkSchemaDemo/data/flight*.csv')
csv_df.show(5)
print(csv_df.schema.simpleString())

# json
json_df = spark.read.format('json').load('/Users/snishad/SparkDemo/04-SparkSchemaDemo/data/flight*.json')
json_df.show(5)
print(json_df.schema.simpleString())

# parquet
parquet_df = spark.read.format('parquet').load('/Users/snishad/SparkDemo/04-SparkSchemaDemo/data/flight*.parquet')
parquet_df.show(5)
print(parquet_df.schema.simpleString())

+--------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
| FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+--------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|1/1/2000|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|1/1/2000|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|1/1/2000|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946|
|1/1/2000|

+--------+---------+------------+------------+--------+----+--------------+--------+--------+----------+-----------------+------+----------------+-------+---------+
|ARR_TIME|CANCELLED|CRS_ARR_TIME|CRS_DEP_TIME|DEP_TIME|DEST|DEST_CITY_NAME|DISTANCE| FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|TAXI_IN|WHEELS_ON|
+--------+---------+------------+------------+--------+----+--------------+--------+--------+----------+-----------------+------+----------------+-------+---------+
|    1348|        0|        1400|        1115|    1113| ATL|   Atlanta, GA|     946|1/1/2000|        DL|             1451|   BOS|      Boston, MA|      5|     1343|
|    1543|        0|        1559|        1315|    1311| ATL|   Atlanta, GA|     946|1/1/2000|        DL|             1479|   BOS|      Boston, MA|      7|     1536|
|    1651|        0|        1721|        1415|    1414| ATL|   Atlanta, GA|     946|1/1/2000|        DL|             1857|   BOS|      Boston, MA|      9|     1642|
|    2005|

In [9]:
# TWO WAYS TO DEFINE SCHEMA 
# PROGRAMATICALLY AND DDL

from pyspark.sql.types import *

# PROGRAMATICALLY
schema = StructType([
        StructField("FL_DATE", DateType()),
        StructField("OP_CARRIER", StringType()),
        StructField("OP_CARRIER_FL_NUM", IntegerType()),
        StructField("ORIGIN", StringType()),
        StructField("ORIGIN_CITY_NAME", StringType()),
        StructField("DEST", StringType()),
        StructField("DEST_CITY_NAME", StringType()),
        StructField("CRS_DEP_TIME", IntegerType()),
        StructField("DEP_TIME", IntegerType()),
        StructField("WHEELS_ON", IntegerType()),
        StructField("TAXI_IN", IntegerType()),
        StructField("CRS_ARR_TIME", IntegerType()),
        StructField("ARR_TIME", IntegerType()),
        StructField("CANCELLED", IntegerType()),
        StructField("DISTANCE", IntegerType())
    ])


# csv
csv_df = spark.read.format('csv').option('inferSchema', 'true').option('header', 'true').schema(schema).option('dateFormat', 'm/d/y').option('mode', 'FAILFAST').load('/Users/snishad/SparkDemo/04-SparkSchemaDemo/data/flight*.csv')
csv_df.show(5)
print(csv_df.schema.simpleString())

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

In [10]:
# SPARK DATA SINK

from pyspark.sql.functions import spark_partition_id

df = spark.read.format('parquet').load('/Users/snishad/SparkDemo/04-SparkSchemaDemo/data/flight*.parquet')

df.groupBy('ORIGIN').count().show()

df = df.repartition(2)

# df.write.format('json').mode('overwrite').save('/Users/snishad/SparkDemo/05-DataSinkDemo/upload/')

# Check the partitions

print(f'Number of partitions = {df.rdd.getNumPartitions()}')

# Count data in each partitions

df.groupBy(spark_partition_id()).count().show()

# write with logical partition by and max records per file
# df.write.format('json').mode('overwrite').partitionBy('OP_CARRIER', 'ORIGIN').option('maxRecordsPerFile', 1000).save('/Users/snishad/SparkDemo/05-DataSinkDemo/upload/')
# df.write.format('json').mode('overwrite').partitionBy('OP_CARRIER', 'ORIGIN').save('/Users/snishad/SparkDemo/05-DataSinkDemo/upload/')


+------+-----+
|ORIGIN|count|
+------+-----+
|   BGM|   93|
|   DLG|   18|
|   MSY| 4150|
|   GEG| 1028|
|   SNA| 3730|
|   BUR| 2252|
|   GTF|  215|
|   GRB|  224|
|   GRR|  771|
|   PSG|   62|
|   EUG|  185|
|   PVD| 2020|
|   GSO| 1265|
|   MYR|  164|
|   OAK| 4597|
|   MSN|  385|
|   FAR|  231|
|   SCC|   44|
|   DCA| 6818|
|   MLU|  181|
+------+-----+
only showing top 20 rows



Number of partitions = 2
+--------------------+------+
|SPARK_PARTITION_ID()| count|
+--------------------+------+
|                   0|235238|
|                   1|235239|
+--------------------+------+



In [11]:
# SPARK TRANSFORMATIONS

from pyspark.sql.functions import column, col

# Four ways to reffer a column in DF

df.select(column('ORIGIN'), col('OP_CARRIER'), df.ORIGIN_CITY_NAME, 'DEST_CITY_NAME').show()




+------+----------+----------------+--------------+
|ORIGIN|OP_CARRIER|ORIGIN_CITY_NAME|DEST_CITY_NAME|
+------+----------+----------------+--------------+
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|        AS|     Oakland, CA|   Seattle, WA|
|   OAK|    

In [16]:
from pyspark.sql.functions import expr

data_list = [("Ravi", "28", "1", "2002"),
                 ("Abdul", "23", "5", "81"),  # 1981
                 ("John", "12", "12", "6"),  # 2006
                 ("Rosy", "7", "8", "63"),  # 1963
                 ("Abdul", "23", "5", "81")]  # 1981


name_df = spark.createDataFrame(data_list).toDF('name', 'day', 'month', 'year')


# Case when then

name_df = name_df.withColumn('year', expr("""
case when year < 21 then year + 2000
when year < 100 then year + 1990
else year
end"""))



+-----+---+-----+------+
| name|day|month|  year|
+-----+---+-----+------+
| Ravi| 28|    1|  2002|
|Abdul| 23|    5|2071.0|
| John| 12|   12|2006.0|
| Rosy|  7|    8|2053.0|
|Abdul| 23|    5|2071.0|
+-----+---+-----+------+

